## Overview
A transaction is a facility provided by databases to combine multiple database read and writes into one logical unit executed atomically - either the entire transaction fails or it completes. It simplifies reasoning about application state since no partial failure happens.

## ACID
Safety guarantees provided by transactions can be consolidated into one acronym - ACID.

### Atomicity
In general programming terminology, atomicity in a multthreaded environment means that operation executed by a thread cannot result in a half-finished state from the point of view of other threads.

Atomicity in ACID is unrelated to concurrency. It means that when multiple writes are made by a client and a fault occurs midway, either the entire transaction is completed or none of it is.

Without atomicity it becomes difficult to determine what change went through and what didn't. Retrying in such scenario can lead to twice the number of changes than expected.

### Consistency
Similar to atomicity, consistency means different things in different contexts. In case of database transaction, it means that there are provisions to enforce invariants about data being stored, thereby ensuring that database remains valid and rule-compliant before and afer transaction.

One way is by declaring *constraints* such as foreign key constraint, uniqueness constraints, triggers, etc. However, it is also application programmers responsibility to define transactions in a way that preserve consistency.

### Isolation
Accounts for scenarios when the database is accessed by multiple clients at the same time. When multiple concurrent writes are happening, there is a possibility of race-condition.

Isolation in context of transaction means that one transaction is isolated from other. Concurrent transactions appear to have executed serially - much that `synchronized` methods.

Isolation has performance penalty, therefore there are different levels of isolation provided by databases. Weaker isolation levels allow concurrent transactions to interfer with each other in limited ways but improve performance.

![Isolation levels](./images/kAIV6an.png)

### Durability
Refers to promise that once a transaction is committed, and data that was written will be retained even in case of failures like database crash. This means that system call like `fsync` is used to ensure that memort buffer is flushed and content is written to the disk

If the database is replicated, this would mean that writes are acknowledged some number of replica nodes.

Perfect durability though, doesn't exist If the disk corrupts and backups get destroyed there is nothing a database can do.

## Isolation Levels
If only read-only statements are involved, there is no concurrency issue. Concurrency issue pops up when:
- one transaction reads data written by other
- two transactions modify the same data

### Read Committed
This isolation level promises:
- No **dirty read**: while reading data from the database, only the data that was committed is read. To simplify, any write made during a transaction is visible to others only when it commits. This ensures that data is not read in partial state. If dirty read was alowed, and the transaction was rolled back, this would mean that rolled back data was read by other process.
- No **dirty write**: while writing to the database, only data that was committed is overwritten. 

![Read Committed](./images/read_committed.png)

Read committed is the default isolation level for most of the databases like Postgres and Oracle. 

**Implementation:** 
1. **dirty write:** *row level locks* are used. Before modifying a row, a transaction must acquire a lock for that row.
2. **dirty read:**
    1. **using locks:** similar to how dirty writes are prevented. This is the default method used by Ms SQL Server
    2. **statement-level MVCC:** database maintains multiple versions of a row. Each transaction is given a consistent view (a "snapshot") of the data at a specific point in time, determined by a transaction ID or timestamp. Note that this is statement level MVCC, so two different select statements within the same transaction, might get different results. Postgres and Oracle utilise this method.

To elaborate, consider the example of a transaction T1 executes:
```sql
UPDATE employee SET department = 'Marketing' WHERE department = 'Sales and Marketing';
```
Lets say that this command identifies three rows to be updated. However, at the same time another transaction T2 executes and inserts a row with the same department name as the above query. That new row would not be visible to T1 since it was not committed before T1 started. What if T2 instead of inserting, updated the rows (changing the department name of two rows). In this case T2 would have acquired lock on the rows, therefore T1 has to wait. Once T1 is able to acquire the lock, it re-checks whether the three rows identified earlier still qualify for the `WHERE` clause. Only the rows that sill match are impacted.

Another example involves two transactions T1 and T2 executing the same set of commands concurrently:
```sql
BEGIN TRANSACTION;
UPDATE accounts SET balance = balance + 100.00 WHERE acctnum = 12345; -- 1
UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 7534;  -- 2
COMMIT;
```
Each `UPDATE` acquires a row-level exclusive lock on the target row. Lets say T1 executes statement 1 first, therefore it acquires lock on row with acctnum=12345. T2 must wait till this lock is released. T1 then acquires lock on row with acctnum=7534. Only after the transaction is complete, both locks are released and T2 can continue execution.  
If T1 and T2 would have executed statements 1 and 2 in different orders, there is a possibility of deadlock. Postgres would detect this and rollback one of the transactions.

**Issues:** there are multiple different possible issues that can popup:
1. **Non-repeatable read:** a transaction reads the same row twice and gets different data each time because another transaction has modified (updated or deleted) the data in the meantime.
   ```sql
   -- Transaction A
   BEGIN TRANSACTION;
   UPDATE employee SET salary = salary * 1.15 WHERE id = 5; -- 1
   COMMIT;
   ```
   ```sql
   -- Transaction B
   START TRANSACTION;
   SELECT AVG(salary) FROM employee; -- 2
   SELECT MAX(salary) FROM employee; -- 3
   COMMIT;
   ```
   If the execution order is `2->1 (+commit)->3` the average and max read is not in sync with each other.

2. **Phantom read:** occurs when a query is run twice with the same criteria, but returns a different set of rows the second time because another transaction has inserted or deleted rows that match the query condition in the meantime.:
   ```sql
   -- Transaction A
   BEGIN TRANSACTION;
   SELECT COUNT(*) FROM appointments WHERE doctor_id = 1 AND status = 'FREE'; -- 1
   SELECT COUNT(*) FROM appointments WHERE doctor_id = 1 AND status = 'FREE'; -- 2
   COMMIT;
   ```
   ```sql
   -- Transaction B
   BEGIN TRANSACTION;
   INSERT INTO appointments (appt_id, doctor_id, appt_time, status) 
       VALUES (3, 1, '2025-07-01 11:00:00', 'FREE'); -- 3
   COMMIT;
   ```
   If the execution order is `1->3 (+commit)->2` then we get different counts for both select.

3. **Lost Updates:** happens when two or more transactions read the same data and then overwrite each other’s updates, causing one transaction’s changes to be silently lost.
   ```sql
   -- Transaction A
   BEGIN TRANSACTION;
   SELECT value FROM metrics WHERE counter_name = 'visits'; -- 1
   -- say the value was 100, now we need to increment it to 101
   UPDATE metrics SET value = 101 WHERE counter_name = 'visits'; -- 2
   COMMIT;
   ```
   ```sql
   -- Transaction B
   BEGIN TRANSACTION;
   SELECT value FROM metrics WHERE counter_name = 'visits'; -- 3
   -- say the value was 100, now we need to increment it to 101
   UPDATE metrics SET value = 101 WHERE counter_name = 'visits'; -- 4
   COMMIT;
   ```
   Instead of value being set to 102, both set it to 101. Essentialy, increment of one transaction is lost.

### Repeatable Read and Snapshot Isolation
As per ANSI specification, `REPEATABLE READ` isolation level guarantees no dirty read and repeatable read. Database like Postgres goes above this, ensuring no phantom reads as well.

**Snapshot Isolation"** provides each transaction a consistent snapshot of the database. During a transaction, the view of database for a transaaction is the same as how it was when the transaction began. Snapshot isolation is implemented using a techniqu called as *MVCC (Multi Version Concurrency Control)*. Under MVCC, each row of the database is tagged with a transaction id. Thus there are multiple versions of the same row, each representing a unique transaction. To be more specific, each version contains a *created by* and a *deleted by* field (for `UPDATE` both the fields are populated).

Contrasting `REPEATABLE READ` with `READ COMMITTED` when discussing example used earlier:
```sql
BEGIN TRANSACTION;
UPDATE accounts SET balance = balance + 100.00 WHERE acctnum = 12345; -- 1
UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 7534;  -- 2
COMMIT
```

If T1 executes first, it obtains lock on the two rows and updates them. Once lock is released after commit, T2 acquires the lock and attempts to udpate the same two rows. In this case however, T2's view of the database doesn't reflect the changes just made by T1. Database like Postgres detects that T2’s view of the world is now invalid (because it missed T1’s committed changes but touched overlapping data). This results in an error being thrown (Postgres returns "ERROR: could not serialize access due to concurrent update") and T2 is rolledback. The ideal scenario would be to retry T2. Which is why the advise is to retry transactions on failure when repeatable read mode is used. *Lost updates* therefore doesn't impact repeatable read mode.

**Issues:**
1. **Write skew:** two concurrent transactions read overlapping data, make decisions based on their snapshots, and then write changes to different rows in a way that even though individually both transactions appear consistent, togetherthe combined changes violate consistency constraint. As as example, lets consider a database that records doctors on-call schedule. The business rule requires that each shift has atleast 1 doctor on-call. If T1 is:
   ```sql
   BEGIN TRANSACTION;
   SELECT COUNT(*) FROM doctors WHERE on_call = true AND shift_id = 5;
   -- Let's say count returned is 2. This is greater than one, so the below statement gets executed
   UPDATE doctors SET on_call = false WHERE doctor_id = 1 AND shift_id = 5;
   COMMIT;
   ```
   The second transaction T2 does the same, but for another doctor:
   ```sql
   BEGIN TRANSACTION;
   SELECT COUNT(*) FROM doctors WHERE on_call = true AND shift_id = 5;
   -- Let's say count returned is 2. This is greater than one, so the below statement gets executed
   UPDATE doctors SET on_call = false WHERE doctor_id = 2 AND shift_id = 5;
   COMMIT;
   ```
   After both transactions complete, there are no doctors on-call for shift id 5, violation business rule. Write skew is a more generalised form of lost update. Unlike lost updates though databases cannot automatically detect this. `SELECT FOR UPDATE` construct can help in this instance. However, there are other examples where (phantom causing write skew) this is more complicated:
   ```sql
   BEGIN TRANSACTION;
   SELECT COUNT(*) FROM bookings WHERE room_id = 12 AND start_time >= '2025-05-04T12:00' AND end_time <= '2025-05-0T13:00';
   -- If previous query returned 0, then proceed with insert statement below:
   INSERT INTO bookings (room_id, start_id, end_time) VALUES (12, '2025-05-04T12:00', '2025-05-0T13:00');
   COMMIT;
   ```
   It is possible in this case that multiple people book the same room at the same time. `SELECT FOR UPDATE` doesn't work here as we are looking for count to be zero, therefore there are no rows to lock!
3. **Long Running Transactions:** since writes require row-level locks that are held until commit, long-running `REPEATABLE READ` transactions can block other transactions wanting to update the same rows.

### Serializable
Strictest isolation level, emulating serial transaction execution for all committed transactions; as if transactions had been executed one after another, serially, rather than concurrently. Serializable mode is implemented in the following ways:  
1. **Serial execution:** run transactions in a single thread.
2. **Two phased locking:** stricter form of locking which allows multiple transactions to read, however writing requires exclusive lock. This means that,
    - If transaction A reads an object and transaction B wants to write to same object, then B must wait till A commits (or aborts)
    - If transaction A has written to an object and transaction B wants to write to it, B must wait till A commits (or aborts)
   Therefore, writers block other writers and readers. This is implmented by having locks operate in two modes -

   ![Lock Modes](./images/lock_mode.png)